# Exploratory Data Analysis (EDA)

Exploratory data analysis is a necessary step to understand the structure of the data, run checks for data issues, produce one and two way analysis to develop hypotheses and inform both feature engineering feature selection.

The question for the analysis is ambiguous - 'use the data set provided to answer questions such as "how likely is a payment to be successful"'. This could be for a payment that has only just been created, or one that has initiated, or even one that has made it to the authorisation stage. Of course if it makes it to being authorised far it will be highly likely to execute. Dataset also includes colummns that will result in clear leakage - e.g `failure_reason` and `executed_at` - these should not be included as predictors.

We could restrict the data to those that have either succeeded or failed, and build a model to **understand** what is driving the probability of failure. Alternatively we could develop it as a predictive model to predict the probability for a given payment, but we won't necessarily have e.g 'time of authorisation' or even 'initiation'. We could say given that they have made it to this point. In that case feature engineering could look like follows:
* Time-to-event as a feature (lastupdate - createddate) for success/failures.
* Initiation time (initiation date - createdate): this will missing if it didn't make it to this stage.
* Authorisation time (authorized date - authorising date) - noting some people won't make it this far impliying failure
* Were they initiated? (intiation date is missing or not) - same as above
* Were they authorised? (authorisation is missing or not) - same as above
- Everyone that settles executes, so using this as a feature results in leakage - do not include as predictor.

The overall goal of the analysis (not in this notebook) is to build a predictive model to predict whether a payment is successful (`executed`) or not. Before jumping into building a model the following should be assessed / inspected:

* Missing values: need to be dealt with appropriately, or they can bias the analysis - particularly if they are not missing at random.
* Feature imbalance: a factor with a category with only a handful of observations is unlikely to 
* Target imbalance: if the outcome is not close to an even split, different evaluation metrics will be required compared to accuracy/ AUC ROC.
* Univariate associations: which features are associated with the outcome, not adjusting for confounding variables
* Feature engineering: what features can we combine to create new features that are predictive of the outcome
* Multicollinearity: highly correlated variables can cause estimation issues, high variance and complicate interpretation. If two variables are perfectly correlated one should be dropped, as it won't offer any more information but will cost degrees of freedom.

## Set up

Load relevant packages

In [1]:
# Import relevant libraries
import pandas as pd
import plotly
import numpy as np

In [2]:
# Load the data
data = pd.read_csv("../data/truelayer_data_sc_test_data_set.csv")

## Inspect Data

In [7]:
# Observe a snippet of the data, displaying the first few rows
pd.set_option('display.max_columns', None)  # None means no limit on the number of columns displayed
from IPython.display import Image
data.head(10)

,id,bank_id,currency,status,api_version,failure_reason,failure_stage,customer_id,vertical,connectivity_type,amount_in_currency,country_id,createdat_ts,lastupdatedat_ts,initiated_at,executed_at,failed_at,authorizing_at,authorized_at,settled_at
0,3c6646ef17a52ac2e35fdb6a15aa44db6d85985b,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,28.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 06:00:06.452249,2010-02-10 06:00:14.766862,2010-02-10 06:00:06.452249,2010-02-10 06:00:14.766862,NaN,2010-02-10 06:00:06.452249,2010-02-10 06:00:06.452249,NaN
1,6c3e58befae5a4880c62c54bdffe7243ab7b66cc,34a0061ba48d1c2810cd930dd69d42482ca92d85,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,18.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-01-12 06:00:17.890703,2010-01-12 06:00:22.517782,2010-01-12 06:00:17.890703,2010-01-12 06:00:22.517782,NaN,2010-01-12 06:00:17.890703,2010-01-12 06:00:17.890703,NaN
2,b2ee452081f287aa12f0efbd05c278edd332def9,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,28.50,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 07:20:57.816799,2010-02-10 07:21:04.284348,2010-02-10 07:20:57.816799,2010-02-10 07:21:04.284348,NaN,2010-02-10 07:20:57.816799,2010-02-10 07:20:57.816799,NaN
3,5aa68c655e0ca1e57bcf4a70c269151811e3ee77,660054a4565377c6e43ff7709abf56d8494ba604,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,16.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-10 06:00:20.172570,2010-02-10 06:00:25.549863,2010-02-10 06:00:20.172570,2010-02-10 06:00:25.549863,NaN,2010-02-10 06:00:20.172570,2010-02-10 06:00:20.172570,NaN
4,1d72b60adefab1066d7421d72a2c3abb3d558544,0090e57640ed78b0c16ac4606a6773769545bb17,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,5.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-08-14 13:42:02.939891,2009-08-14 13:42:54.515245,2009-08-14 13:42:09.978416,2009-08-14 13:42:54.515245,NaN,2009-08-14 13:42:03.699860,2009-08-14 13:42:52.858713,2009-08-14 13:42:54.515245
5,bcfa97793076f65649b957dd799997ba2efaef3b,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,100.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-08-20 14:05:30.662267,2009-08-20 14:06:04.153539,2009-08-20 14:05:35.397045,2009-08-20 14:06:03.968660,NaN,2009-08-20 14:05:31.590486,2009-08-20 14:06:02.119292,2009-08-20 14:06:04.153539
6,caf692e3132a3ede4bce389283bbe578229018b4,6926610db1dfc17aca46a6f67789d7d6c24a7185,CAD,failed,v3,expired,initiated,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,7.08,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-09-05 01:37:49.441806,2009-09-05 01:52:49.567983,2009-09-05 01:37:54.902954,NaN,2009-09-05 01:52:49.567983,2009-09-05 01:37:49.926388,NaN,NaN
7,f23ca43a1cfc1ed47ecb6c37dd352ff426644d27,b5ab8f51d35b64af079485e5bbbc335213f3a0ae,CAD,Executed,v3,NaN,NaN,365a3d703f257f52adde00af51c7f4897a6f9c6a,vertical 2,type 8,40.50,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-01-19 06:00:22.670235,2010-01-19 06:00:28.520191,2010-01-19 06:00:22.670235,2010-01-19 06:00:28.520191,NaN,2010-01-19 06:00:22.670235,2010-01-19 06:00:22.670235,NaN
8,d5478919b341d23f0cba6cf31e20728e71afc257,4d762ce64874cd49be593313e92a1e61cf74417e,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,50.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2009-10-22 15:17:17.046271,2009-10-22 15:17:56.370269,2009-10-22 15:17:23.534314,2009-10-22 15:17:56.222319,NaN,2009-10-22 15:17:18.082226,2009-10-22 15:17:56.035144,2009-10-22 15:17:56.370269
9,da1be4c92610de8116f34d30c57635b26c40af1a,e8085010136a26d0ebc200ac0751b1d9d673a764,CAD,Executed,v3,NaN,NaN,64c370a753b1db282770cdbf2aba5434a3185b3d,vertical 2,type 8,20.00,9e2b06736b477b7a924f60de14a7e329d82d6f4f,2010-02-05 05:06:16.494003,2010-02

From a first glance, `status` is likely the target of the analysis. Now inspect further to see if it requires dichotomising:

In [5]:
data['status'].value_counts(normalize= True)
print(data['customer_id'].nunique())

173


Feature engineer the target variable to `Executed` vs `Other`, noting that the target is **imbalanced**. Restrict to statuses that aren't in progress to simplify problem, noting this increases imbalance.

In [7]:
data = data.loc[data["status"].isin(['Executed', 'failed', 'Failed', 'Cancelled', 'Rejected']), :].copy()
data['outcome'] = np.where(data['status'] == 'Executed', 'Executed', 'Other') # Dichotomise target
data['outcome'].value_counts(normalize=True) # Check % aligns with what we would expect having seen value_counts above.

outcome
Executed    0.847488
Other       0.152512
Name: proportion, dtype: float64

Inspecting the data further below, only a single feature (`amount_in_currency`) is numeric.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 445571 entries, 0 to 499999
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  445571 non-null  object 
 1   bank_id             445571 non-null  object 
 2   currency            445571 non-null  object 
 3   status              445571 non-null  object 
 4   api_version         445571 non-null  object 
 5   failure_reason      44628 non-null   object 
 6   failure_stage       44628 non-null   object 
 7   customer_id         445571 non-null  object 
 8   vertical            445565 non-null  object 
 9   connectivity_type   434948 non-null  object 
 10  amount_in_currency  445571 non-null  float64
 11  country_id          445571 non-null  object 
 12  createdat_ts        445571 non-null  object 
 13  lastupdatedat_ts    445571 non-null  object 
 14  initiated_at        232325 non-null  object 
 15  executed_at         377876 non-null  ob

There are missing values for many categorical features. These can be easily treated by creating a new 'Missing' category. First convert the date columns to datetime and then fill the categorical missing values with 'Missing'. Tree based ML model can handle missing data in continuous predictors.

In [9]:
datetime_columns = ['createdat_ts', 'initiated_at', 'authorizing_at', 'authorized_at', 'settled_at', 'lastupdatedat_ts', 'executed_at', 'failed_at']
for col in datetime_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')  # Coerce errors to NaT


object_cols = data.select_dtypes(include='object').columns

# object cols is a list of categorical columns - fill with 'Missing'
data[object_cols] = data[object_cols].fillna('Missing')

# Now check for missing values again
print(data.isna().sum())

id                         0
bank_id                    0
currency                   0
status                     0
api_version                0
failure_reason             0
failure_stage              0
customer_id                0
vertical                   0
connectivity_type          0
amount_in_currency         0
country_id                 0
createdat_ts               0
lastupdatedat_ts           0
initiated_at          213246
executed_at            67695
failed_at             400943
authorizing_at        211205
authorized_at         247609
settled_at            372211
outcome                    0
dtype: int64


In [10]:
data.describe(include = 'all')

,id,bank_id,currency,status,api_version,failure_reason,failure_stage,customer_id,vertical,connectivity_type,amount_in_currency,country_id,createdat_ts,lastupdatedat_ts,initiated_at,executed_at,failed_at,authorizing_at,authorized_at,settled_at,outcome
count,445571,445571,445571,445571,445571,445571,445571,445571,445571,445571,4.455710e+05,445571,445571,445571,232325,377876,44628,234366,197962,73360,445571
unique,445571,185,4,5,3,21,6,164,6,7,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,3c6646ef17a52ac2e35fdb6a15aa44db6d85985b,6cd7d8d37b110668afdca42fc0e1d366b322783e,CAD,Executed,v3,Missing,Missing,63237c0742e42bfd7d59d68d894d010aa4cca170,vertical 2,type 8,NaN,9e2b06736b477b7a924f60de14a7e329d82d6f4f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Executed
freq,1,60106,394239,377616,236176,400943,400943,128733,305751,408876,NaN,394408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377616
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.221611e+01,NaN,2009-02-07 00:39:36.786365184,2009-02-07 01:40:06.038881280,2009-06-11 20:54:09.054828544,2009-02-09 05:02:27.730088704,2009-05-26 23:38:23.565870080,2009-06-11 21:06:33.975676160,2009-06-16 07:46:21.638830848,2009-07-25 20:59:48.465589760,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,2004-10-20 15:28:02.026496,2004-10-20 15:28:05.911291,2007-09-30 23:31:41.808812,2005-05-28 08:48:39.897626,2007-09-14 21:27:13.530686,2007-09-14 21:25:58.764009,2007-09-30 23:31:41.808812,2007-10-02 06:08:05.764732,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.500000e+00,NaN,2008-08-31 13:11:12.024657408,2008-08-31 14:40:09.302836992,2009-02-03 10:13:25.695742976,2008-09-05 17:58:15.772144640,2009-01-02 08:27:21.757950208,2009-02-03 09:22:49.830285568,2009-02-12 09:39:41.953563136,2009-04-17 19:04:37.217715456,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.950000e+00,NaN,2009-05-14 18:02:27.711470080,2009-05-14 18:26:55.093530880,2009-08-02 18:31:26.774753024,2009-05-18 03:50:28.667093504,2009-07-23 09:53:57.448796672,2009-08-02 18:13:23.063495424,2009-08-05 11:12:24.585685504,2009-09-22 18:21:25.595727360,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.500000e+01,NaN,2009-10-21 04:49:01.966942464,2009-10-21 05:11:22.840642048,2009-11-20 11:06:43.505166080,2009-10-22 17:05:30.067226880,2009-11-13 18:09:12.897237248,2009-11-20 10:12:05.443002880,2009-11-21 17:39:04.785843456,2009-12-11 13:14:44.935749888,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.266876e+06,NaN,2010-02-11 00:01:05.197460,2010-02-11 00:02:10.047000,2010-02-11 00:01:09.664104,2010-02-11 00:02:07.390437,2010-02-10 22:42:01.505789,2010-02-11 00:01:07.854736,2010-02-11 00:02:05.006278,2010-02-11 00:02:10.047000,NaN


* `id` is unique, meaning each row is unique. This is a redundant variable.
* There are 173 unique customers acros 50,000 payments and 203 banks. Categorical features with a large number of unique categories require special attention, as including them will result in a much larger number of parameters - high cardinality.
* `failure_reason` and `failure_stage` occur after the outcome of interest, these cannot be used as features as they would not be available in new data to perform the prediction.
* `amount_in_currency` ranges from 0 to 1.3 million.

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 445558 entries, 0 to 499999
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  445558 non-null  object        
 1   bank_id             445558 non-null  object        
 2   currency            445558 non-null  object        
 3   status              445558 non-null  object        
 4   api_version         445558 non-null  object        
 5   failure_reason      445558 non-null  object        
 6   failure_stage       445558 non-null  object        
 7   customer_id         445558 non-null  object        
 8   vertical            445558 non-null  object        
 9   connectivity_type   445558 non-null  object        
 10  amount_in_currency  445558 non-null  float64       
 11  country_id          445558 non-null  object        
 12  createdat_ts        445558 non-null  datetime64[ns]
 13  lastupdatedat_ts    445558 non-nul

In [17]:
data['duration'] = (data['lastupdatedat_ts'] - data['createdat_ts']).dt.total_seconds() # Duration in seconds
data.head()
data['duration'].describe(include='all') # Check the distribution of duration

count    4.455710e+05
mean     3.629253e+03
std      2.298210e+05
min     -8.630456e+04
25%      2.489476e+01
50%      3.666249e+01
75%      5.842416e+01
max      8.321838e+07
Name: duration, dtype: float64

Duration is highly skewed, most data points lie between 2 and 6 seconds, but a material number are much longer. Fortunately, tree based ML models are robust to outliers. There are also a handful negative durations which need to be dealt with - these are assumed to be timestamp errors and are dropped.

In [22]:
print(data.query("duration < 0").shape)
data = data.query("duration >= 0").copy()

(0, 22)


Most of the time columns will result in leakage, as the model will figure out e.g someone that has not been initiated or authorised has not been executed. It does not make sense to include these as predictors. The following visualises the count and proportions for various candidate predictors:

In [28]:
import plotly.graph_objects as go
import pandas as pd

# Function to create dual-axis bar and line plots for categorical columns
def plot_categorical_relationships_dual_axis(data, categorical_columns, outcome):
    for col in categorical_columns:
        if data[col].nunique() <= 30:  # Limit to a maximum of 10 unique categories for better readability
            # Create a new DataFrame for the plot
            plot_data = data.groupby([col, outcome]).size().reset_index(name='count')

            # Calculate proportions
            total_counts = plot_data.groupby(col)['count'].sum().reset_index(name='total_count')
            plot_data = plot_data.merge(total_counts, on=col)
            plot_data['proportion'] = plot_data['count'] / plot_data['total_count']

            # Create the figure
            fig = go.Figure()

            # Add bars for count
            for outcome_value in plot_data[outcome].unique():
                filtered_data = plot_data[plot_data[outcome] == outcome_value]
                fig.add_trace(go.Bar(
                    x=filtered_data[col],
                    y=filtered_data['count'],
                    name=f'Count: {outcome_value}',
                    text=filtered_data['count'],
                    textposition='auto'
                ))

            # Add line for proportion on the secondary y-axis
            fig.add_trace(go.Scatter(
                x=filtered_data[col],
                y=filtered_data['proportion'],
                name='Proportion',
                mode='lines+markers',
                yaxis='y2',
                line=dict(color='royalblue', width=2),
                text=filtered_data['proportion'].round(2),
                textposition='top center'
            ))

            # Update layout
            fig.update_layout(
                title=f'Relationship between {col} and {outcome}',
                xaxis_title=col,
                yaxis_title='Count',
                yaxis2=dict(title='Proportion', overlaying='y', side='right', showgrid=False),
                barmode='group',
                template='plotly_white'
            )

            fig.show()

# List of categorical columns (replace this with your actual object columns list)
categorical_cols = [
    'currency',
    'status',
    'api_version',
    'failure_reason',
    'failure_stage',
    'vertical',
    'connectivity_type',
    'country_id'
]

# Call the function
plot_categorical_relationships_dual_axis(data, categorical_cols, 'outcome')


* Currency has sparse categories that will need to be feature engineered to combine GBP and Euros.
* Failure stage demonstrates the leakage issue - everyone who executed has 'Missing'. The model would use this to make predictions if this was included, but this wouldn't be available for a new prediction until we already know the outcome.
* Combine vertical 2, 3 and 4 into a single category.
* Take the 4 most credible countries and combine the rest into an 'other' category.

In [41]:
data['connectivity_type'].value_counts()

connectivity_type
type 8     408872
Missing     10618
type 1       8999
type 6       6444
type 7       4536
type 2       4128
type 3       1961
Name: count, dtype: int64

In [39]:
# Combine GBP and EUR into a single category 'EUR_GBP'
data['currency_group'] = data['currency'].replace({'GBP': 'EUR_GBP', 'EUR': 'EUR_GBP'})

# Check the new distribution
#print(data['currency_group'].value_counts())

# Group smaller categories into 'other_vertical'
data['vertical_group'] = data['vertical'].replace({
    'vertical 1': 'other_vertical',
    'vertical 3': 'other_vertical',
    'Missing': 'other_vertical'
})

# Check the new distribution
print(data['vertical_group'].value_counts())

data['vertical'].value_counts()

# Get the frequency of each country_id
country_counts = data['country_id'].value_counts()

# Define a threshold and mark less frequent categories as 'other_country'
common_countries = country_counts[country_counts > 3500].index
data['country_group'] = data['country_id'].where(data['country_id'].isin(common_countries), 'other_country')

data['country_group'].value_counts()

vertical_group
vertical 2        305741
vertical 5        127505
vertical 4         11357
other_vertical       955
Name: count, dtype: int64


country_group
9e2b06736b477b7a924f60de14a7e329d82d6f4f    394405
7ec009376b8bf0c03561b3f6ba251ab9d96efd42     14636
other_country                                12853
37eec10a53dd5fd2f024a17be5961dfceb0c66e6     10268
f69ad584c025f9b756ce1e8c70de3a33aef50b32      9796
a68db3fc75fd72a12bb2ad1442ff9927bd04139d      3600
Name: count, dtype: int64

In [33]:
features = ['bank_id', 'currency', 'api_version', 'customer_id', 'vertical', 'connectivity_type', 'amount_in_currency', 'country_id', 'createdat_ts', 'initiated_at']

In [52]:
data['created_day_of_week'] = data['createdat_ts'].dt.day_name() 
predictors = ['currency_group', 'api_version', 'vertical_group', 'connectivity_type', 'country_group', 'amount_in_currency', 'duration', 'created_day_of_week', 'createdat_ts']
target = 'outcome'

cols = predictors + [target]
cols
#data[predictors].head()

['currency_group',
 'api_version',
 'vertical_group',
 'connectivity_type',
 'country_group',
 'amount_in_currency',
 'duration',
 'created_day_of_week',
 'createdat_ts',
 'outcome']